In [1]:
import json
import time
import pymongo
import pandas as pd
from datetime import datetime
from kubernetes import client,config,watch

In [2]:
config.load_kube_config()
usage_api = client.CustomObjectsApi()
request_api = client.CoreV1Api()

In [3]:
mongoclient = pymongo.MongoClient("mongodb://localhost:27017/")
mongoclient.drop_database('resource')
db = mongoclient['resource']

In [4]:
for i in range(600):
    timeline = datetime.timestamp(datetime.now())
    
    usage = usage_api.list_namespaced_custom_object(group="metrics.k8s.io",version="v1beta1", namespace="default", plural="pods")['items']
    for i in usage:
#         collection = db[i['metadata']['name']]
        collection = db['usage']
        item = dict({'time':timeline}, **i['containers'][0]['usage'])
        item.update({'pod_name':i['metadata']['name']})
        collection.insert_one(item)
        
    request = request_api.list_namespaced_pod(namespace='default').items
    for i in request:
#         collection = db[i.metadata.name+'-requests']
        collection = db['requests']
        if i.spec.containers[0].resources.requests != None:
            item = dict({'time':timeline}, **i.spec.containers[0].resources.requests)
            item.update({'pod_name':i.metadata.name})
            collection.insert_one(item)
        else:
            item = dict({'time':timeline}, **{'cpu':0, 'memory':0})
            item.update({'pod_name':i.metadata.name})
            collection.insert_one(item)
    time.sleep(1)

In [5]:
# data = usage_api.list_namespaced_custom_object(group="metrics.k8s.io",version="v1beta1", namespace="default", plural="pods")['items']
# for i in data:
#     print(i['metadata']['name'])

In [6]:
# data[1]

In [7]:
# pod = request_api.list_namespaced_pod(namespace='default')
# print(pod.items[0].metadata.name)
# print(pod.items[0].spec.containers[0])

In [8]:
# pod.items[1].spec.containers[0]